In [2]:
import pandas as pd
df_X=pd.read_csv('/home/rliuaj/balance_sheet/df_X')
df_X=df_X.iloc[:,1:]
columns=df_X.columns

df_text = pd.DataFrame()



In [6]:
df_X

,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity,Market Cap,Industry,Employees,City,EBITDA Margins,Profit Margins,Gross Margins,Day High
0,3.969000e+09,9.490000e+09,3.285000e+09,2.881000e+09,7.293000e+09,34573586432,Electronic Gaming & Multimedia,13700.0,Redwood City,0.25510,0.14118,0.78567,133.710
1,4.151000e+09,9.649000e+09,3.513000e+09,2.662000e+09,7.625000e+09,34573586432,Electronic Gaming & Multimedia,13700.0,Redwood City,0.25510,0.14118,0.78567,133.710
2,7.213000e+09,6.075000e+09,2.964000e+09,2.484000e+09,7.840000e+09,34573586432,Electronic Gaming & Multimedia,13700.0,Redwood City,0.25510,0.14118,0.78567,133.710
3,6.563300e+10,1.626420e+11,6.942100e+10,8.708500e+10,7.146900e+10,78612873216,Healthcare Plans,219000.0,Woonsocket,0.03843,0.01366,0.13809,64.050
4,6.000800e+10,1.729910e+11,6.780700e+10,8.981100e+10,7.507500e+10,78612873216,Healthcare Plans,219000.0,Woonsocket,0.03843,0.01366,0.13809,64.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,1.915000e+08,1.021900e+09,7.150000e+07,5.057000e+08,6.362000e+08,629501568,Packaging & Containers,800.0,Concord Township,0.12842,-0.05645,0.37398,7.855
3812,1.072000e+08,1.026300e+09,5.750000e+07,5.531000e+08,5.229000e+08,629501568,Packaging & Containers,800.0,Concord Township,0.12842,-0.05645,0.37398,7.855
3813,2.509420e+08,3.986870e+08,1.100720e+08,1.205690e+08,4.189880e+08,990825856,Specialty Industrial Machinery,1416.0,Austin,0.18731,0.08894,0.42754,29.825
3814,2.444380e+08,3.922310e+08,8.099800e+07,1.564670e+08,3.992040e+08,990825856,Specialty Industrial Machinery,1416.0,Austin,0.18731,0.08894,0.42754,29.825


In [3]:
columns

Index(['Current Assets', 'Total Non Current Assets', 'Current Liabilities',
       'Total Non Current Liabilities Net Minority Interest',
       'Stockholders Equity', 'Market Cap', 'Industry', 'Employees', 'City',
       'EBITDA Margins', 'Profit Margins', 'Gross Margins', 'Day High'],
      dtype='object')

In [4]:
count=1
for ind in df_X.index:
    #try:
    text = "As a financial analyst, you are tasked with estimating next year's balance sheet items for a company, based on the current year's performance of the following items:"
    for i in columns:
        text += f"{i} is {df_X.loc[ind,i]},"
    text+='.\n'

In [5]:
text

"As a financial analyst, you are tasked with estimating next year's balance sheet items for a company, based on the current year's performance of the following items:Current Assets is 206809000.0,Total Non Current Assets is 411069000.0,Current Liabilities is 52798000.0,Total Non Current Liabilities Net Minority Interest is 186208000.0,Stockholders Equity is 378872000.0,Market Cap is 990825856,Industry is Specialty Industrial Machinery,Employees is 1416.0,City is Austin,EBITDA Margins is 0.18731001,Profit Margins is 0.08894,Gross Margins is 0.42754,Day High is 29.825,.\n"

In [11]:
from llm2vec import LLM2Vec

import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftModel

# Loading base Mistral model, along with custom code that enables bidirectional connections in decoder-only LLMs.
tokenizer = AutoTokenizer.from_pretrained(
    "McGill-NLP/LLM2Vec-Llama-2-7b-chat-hf-mntp"
)
config = AutoConfig.from_pretrained(
    "McGill-NLP/LLM2Vec-Llama-2-7b-chat-hf-mntp", trust_remote_code=True
)
model = AutoModel.from_pretrained(
    "McGill-NLP/LLM2Vec-Llama-2-7b-chat-hf-mntp",
    trust_remote_code=True,
    config=config,
    torch_dtype=torch.bfloat16,
    device_map="cuda" if torch.cuda.is_available() else "cpu",
)

# Loading MNTP (Masked Next Token Prediction) model.
model = PeftModel.from_pretrained(
    model,
    "McGill-NLP/LLM2Vec-Llama-2-7b-chat-hf-mntp",
)

# Wrapper for encoding and pooling operations
l2v = LLM2Vec(model, tokenizer, pooling_mode="mean", max_length=512)

# Encoding queries using instructions
instruction = (
    "Given a web search query, retrieve relevant passages that answer the query:"
)
queries = [
    [instruction, "how much protein should a female eat"],
    [instruction, "summit define"],
]
q_reps = l2v.encode(queries)

# Encoding documents. Instruction are not required for documents
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments.",
]
d_reps = l2v.encode(documents)

# Compute cosine similarity
q_reps_norm = torch.nn.functional.normalize(q_reps, p=2, dim=1)
d_reps_norm = torch.nn.functional.normalize(d_reps, p=2, dim=1)
cos_sim = torch.mm(q_reps_norm, d_reps_norm.transpose(0, 1))

print(cos_sim)
"""
tensor([[0.7277, 0.5421],
        [0.4818, 0.5551]])
"""


/home/rliuaj/miniconda3/envs/tf210/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/5d/a8/5da8fea2c104c9664008296f364800e61ac93deaebbb3d5b40b40b8689613b68/9e556afd44213b6bd1be2b850ebbbd98f5481437a8021afaf58ee7fb1818d347?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tokenizer.model%3B+filename%3D%22tokenizer.model%22%3B&Expires=1739599524&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTU5OTUyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzVkL2E4LzVkYThmZWEyYzEwNGM5NjY0MDA4Mjk2ZjM2NDgwMGU2MWFjOTNkZWFlYmJiM2Q1YjQwYjQwYjg2ODk2MTNiNjgvOWU1NTZhZmQ0NDIxM2I2YmQxYmUyYjg1MGViYmJkOThmNTQ4MTQzN2E4MDIxYWZhZjU4ZWU3ZmIxODE4ZDM0Nz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=nRvmEXVQQRNicriR3tm3V73OBSAsOp6R1jeiYXJLieWmRQzICrZcOMBaya4X%7EjpF5asPu7sskfe1oG0-guoE8uYwGSXj9GdUIB54XuoN7JZrTvyBVhMkiuEHUUSLZ%7EunXe0Q62Fs-wv0cUJpSsKKT4%7EoxV5tc4cW7YotQ8A8lG8JHU-xOxrAYJKce-iSORCxGn1HvtvOkkcv3XkJd06derJTcWNe0GCszZask2GBgsMdut2QT%7EfZeINwo86OymfH

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/5d/a8/5da8fea2c104c9664008296f364800e61ac93deaebbb3d5b40b40b8689613b68/9e556afd44213b6bd1be2b850ebbbd98f5481437a8021afaf58ee7fb1818d347?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tokenizer.model%3B+filename%3D%22tokenizer.model%22%3B&Expires=1739599524&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTU5OTUyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzVkL2E4LzVkYThmZWEyYzEwNGM5NjY0MDA4Mjk2ZjM2NDgwMGU2MWFjOTNkZWFlYmJiM2Q1YjQwYjQwYjg2ODk2MTNiNjgvOWU1NTZhZmQ0NDIxM2I2YmQxYmUyYjg1MGViYmJkOThmNTQ4MTQzN2E4MDIxYWZhZjU4ZWU3ZmIxODE4ZDM0Nz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=nRvmEXVQQRNicriR3tm3V73OBSAsOp6R1jeiYXJLieWmRQzICrZcOMBaya4X%7EjpF5asPu7sskfe1oG0-guoE8uYwGSXj9GdUIB54XuoN7JZrTvyBVhMkiuEHUUSLZ%7EunXe0Q62Fs-wv0cUJpSsKKT4%7EoxV5tc4cW7YotQ8A8lG8JHU-xOxrAYJKce-iSORCxGn1HvtvOkkcv3XkJd06derJTcWNe0GCszZask2GBgsMdut2QT%7EfZeINwo86OymfH

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/5d/a8/5da8fea2c104c9664008296f364800e61ac93deaebbb3d5b40b40b8689613b68/9e556afd44213b6bd1be2b850ebbbd98f5481437a8021afaf58ee7fb1818d347?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tokenizer.model%3B+filename%3D%22tokenizer.model%22%3B&Expires=1739599524&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczOTU5OTUyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzVkL2E4LzVkYThmZWEyYzEwNGM5NjY0MDA4Mjk2ZjM2NDgwMGU2MWFjOTNkZWFlYmJiM2Q1YjQwYjQwYjg2ODk2MTNiNjgvOWU1NTZhZmQ0NDIxM2I2YmQxYmUyYjg1MGViYmJkOThmNTQ4MTQzN2E4MDIxYWZhZjU4ZWU3ZmIxODE4ZDM0Nz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=nRvmEXVQQRNicriR3tm3V73OBSAsOp6R1jeiYXJLieWmRQzICrZcOMBaya4X%7EjpF5asPu7sskfe1oG0-guoE8uYwGSXj9GdUIB54XuoN7JZrTvyBVhMkiuEHUUSLZ%7EunXe0Q62Fs-wv0cUJpSsKKT4%7EoxV5tc4cW7YotQ8A8lG8JHU-xOxrAYJKce-iSORCxGn1HvtvOkkcv3XkJd06derJTcWNe0GCszZask2GBgsMdut2QT%7EfZeINwo86OymfH

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs-us-1.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: bab8adec-267a-4458-8c7d-2aebd14d78aa)')

In [7]:
import pandas as pd
df_X=pd.read_csv('/home/rliuaj/balance_sheet/df_X')
df_X=df_X.iloc[:,1:]
columns=df_X.columns

df_text = pd.DataFrame()

indx=[]
text_all=[]

count=1


In [10]:

for ind in df_X.index:
    try:
        text = "As a financial analyst, you are tasked with estimating next year's balance sheet items for a company, based on the current year's performance of the following items:"
        for i in columns:
            text += f"{i} is {df_X.loc[ind,i]},"
        text+='.\n'
        # indx.append(ind)
        # text_all.append(text)
        df_text = df_text.append({"indx": ind, "text": text}, ignore_index=True)
        count+=1
        if count%5==0:
            df_text.to_csv("text_all.csv", index=False)
        print('1')
    except:
        pass

In [11]:
df_text

""


In [12]:
text

"As a financial analyst, you are tasked with estimating next year's balance sheet items for a company, based on the current year's performance of the following items:Current Assets is 206809000.0,Total Non Current Assets is 411069000.0,Current Liabilities is 52798000.0,Total Non Current Liabilities Net Minority Interest is 186208000.0,Stockholders Equity is 378872000.0,Market Cap is 990825856,Industry is Specialty Industrial Machinery,Employees is 1416.0,City is Austin,EBITDA Margins is 0.18731001,Profit Margins is 0.08894,Gross Margins is 0.42754,Day High is 29.825,.\n"

In [13]:
df_text = df_text.append({"indx": ind, "text": text}, ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
import pandas as pd

# Create an empty DataFrame to store the results
df_text = pd.DataFrame(columns=["indx", "text"])

# Lists to store data temporarily
indx = []
text_all = []

count = 1

# Loop through df_X rows
for ind in df_X.index:
    try:
        # Start with a base text
        text = "As a financial analyst, you are tasked with estimating next year's balance sheet items for a company, based on the current year's performance of the following items:"

        # Add feature values to the text
        for i in columns:
            text += f" {i} is {df_X.loc[ind, i]},"
        text += '.\n'

        # Append data to the lists (for better performance)
        indx.append(ind)
        text_all.append(text)

        # After every 5 iterations, store data in the DataFrame and write to CSV
        if count % 5 == 0:
            df_text = pd.DataFrame({'indx': indx, 'text': text_all})
            df_text.to_csv("text_all.csv", index=False)

        count += 1
        print(f"Processed {count} rows")

    except Exception as e:
        print(f"Error processing row {ind}: {e}")
        continue

# Final save (in case there are fewer than 5 iterations left)
if len(indx) > 0:
    df_text = pd.DataFrame({'indx': indx, 'text': text_all})
    df_text.to_csv("text_all.csv", index=False)



Processed 2 rows
Processed 3 rows
Processed 4 rows
Processed 5 rows
Processed 6 rows
Processed 7 rows
Processed 8 rows
Processed 9 rows
Processed 10 rows
Processed 11 rows
Processed 12 rows
Processed 13 rows
Processed 14 rows
Processed 15 rows
Processed 16 rows
Processed 17 rows
Processed 18 rows
Processed 19 rows
Processed 20 rows
Processed 21 rows
Processed 22 rows
Processed 23 rows
Processed 24 rows
Processed 25 rows
Processed 26 rows
Processed 27 rows
Processed 28 rows
Processed 29 rows
Processed 30 rows
Processed 31 rows
Processed 32 rows
Processed 33 rows
Processed 34 rows
Processed 35 rows
Processed 36 rows
Processed 37 rows
Processed 38 rows
Processed 39 rows
Processed 40 rows
Processed 41 rows
Processed 42 rows
Processed 43 rows
Processed 44 rows
Processed 45 rows
Processed 46 rows
Processed 47 rows
Processed 48 rows
Processed 49 rows
Processed 50 rows
Processed 51 rows
Processed 52 rows
Processed 53 rows
Processed 54 rows
Processed 55 rows
Processed 56 rows
Processed 57 rows


In [16]:
df_X.shape

(3816, 13)

In [17]:
text_all=pd.read_csv('/home/rliuaj/balance_sheet/text_all.csv')

In [18]:
text_all.shape

(3816, 2)

In [19]:
emb=pd.read_csv('/home/rliuaj/balance_sheet/text_embeddings.csv')

In [20]:
emb

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-0.021312,-0.001856,-0.002905,-0.002381,0.001800,0.013943,0.004421,-0.017458,0.022672,-0.017458,...,-0.023693,-0.020405,0.017231,-0.020859,-0.010939,0.010259,0.015304,0.008219,-0.003273,0.006207
1,-0.021274,-0.002888,-0.004861,-0.002445,0.000515,0.011838,0.004089,-0.017728,0.023790,-0.018186,...,-0.026192,-0.024362,0.017957,-0.020359,-0.008864,0.010580,0.014983,0.007434,-0.003002,0.006834
2,-0.022515,-0.002314,-0.002943,-0.003329,0.002443,0.014115,0.002372,-0.017715,0.025029,-0.017372,...,-0.023315,-0.021944,0.016686,-0.020801,-0.011486,0.008629,0.013143,0.009086,-0.002729,0.004372
3,-0.016384,-0.002162,-0.009500,-0.004665,-0.002844,0.007396,0.004693,-0.012914,0.027648,-0.013938,...,-0.024007,-0.017863,0.016839,-0.017294,-0.004466,0.012174,0.021504,0.013369,-0.008704,0.004551
4,-0.013763,0.004408,-0.007735,-0.004436,-0.002062,0.006227,0.004664,-0.017517,0.027526,-0.012910,...,-0.022066,-0.016948,0.012796,-0.016720,-0.007251,0.011659,0.021156,0.014445,-0.007052,0.005886
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,-0.009266,-0.000375,0.003028,-0.014736,-0.001437,0.005303,0.005638,-0.014401,0.018197,-0.015964,...,-0.026681,-0.017081,0.013117,-0.017974,-0.011052,0.013341,0.014513,0.013396,-0.003991,-0.000352
3812,-0.013146,0.001266,0.002979,-0.014321,-0.003091,0.005818,0.007049,-0.017678,0.015776,-0.017230,...,-0.027636,-0.017566,0.011524,-0.016335,-0.010237,0.014881,0.017566,0.012643,-0.004475,0.001832
3813,-0.019782,0.008103,-0.006762,-0.007432,0.001732,0.008773,0.010785,-0.018106,0.022465,-0.016988,...,-0.022465,-0.022353,0.011735,-0.018888,-0.005616,0.016988,0.020229,0.014976,-0.005225,0.002864
3814,-0.018571,0.002699,-0.005818,-0.008055,0.001727,0.004447,0.008335,-0.017453,0.021368,-0.017341,...,-0.024836,-0.020809,0.011020,-0.017453,-0.007719,0.017117,0.021256,0.013313,-0.006657,0.002335


In [21]:
X=pd.read_csv('/home/rliuaj/balance_sheet/df_X')
X

,Unnamed: 0,Current Assets,Total Non Current Assets,Current Liabilities,Total Non Current Liabilities Net Minority Interest,Stockholders Equity,Market Cap,Industry,Employees,City,EBITDA Margins,Profit Margins,Gross Margins,Day High
0,2023-03-31,3.969000e+09,9.490000e+09,3.285000e+09,2.881000e+09,7.293000e+09,34573586432,Electronic Gaming & Multimedia,13700.0,Redwood City,0.25510,0.14118,0.78567,133.710
1,2022-03-31,4.151000e+09,9.649000e+09,3.513000e+09,2.662000e+09,7.625000e+09,34573586432,Electronic Gaming & Multimedia,13700.0,Redwood City,0.25510,0.14118,0.78567,133.710
2,2021-03-31,7.213000e+09,6.075000e+09,2.964000e+09,2.484000e+09,7.840000e+09,34573586432,Electronic Gaming & Multimedia,13700.0,Redwood City,0.25510,0.14118,0.78567,133.710
3,2022-12-31,6.563300e+10,1.626420e+11,6.942100e+10,8.708500e+10,7.146900e+10,78612873216,Healthcare Plans,219000.0,Woonsocket,0.03843,0.01366,0.13809,64.050
4,2021-12-31,6.000800e+10,1.729910e+11,6.780700e+10,8.981100e+10,7.507500e+10,78612873216,Healthcare Plans,219000.0,Woonsocket,0.03843,0.01366,0.13809,64.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3811,2021-12-31,1.915000e+08,1.021900e+09,7.150000e+07,5.057000e+08,6.362000e+08,629501568,Packaging & Containers,800.0,Concord Township,0.12842,-0.05645,0.37398,7.855
3812,2020-12-31,1.072000e+08,1.026300e+09,5.750000e+07,5.531000e+08,5.229000e+08,629501568,Packaging & Containers,800.0,Concord Township,0.12842,-0.05645,0.37398,7.855
3813,2023-03-31,2.509420e+08,3.986870e+08,1.100720e+08,1.205690e+08,4.189880e+08,990825856,Specialty Industrial Machinery,1416.0,Austin,0.18731,0.08894,0.42754,29.825
3814,2022-03-31,2.444380e+08,3.922310e+08,8.099800e+07,1.564670e+08,3.992040e+08,990825856,Specialty Industrial Machinery,1416.0,Austin,0.18731,0.08894,0.42754,29.825
